In [ ]:
#Import Dependecies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import censusdata
import csv

# Census API Key
from goober import (api_key) 

years = [2009, 2010, 2011, 2012, 2013, 2014, 2015]

In [ ]:
#Create an Empty List for Census DFs
census_data = []

#Query Census Wrapper for given years for parameters
for year in years:
    c = Census(api_key, year=year)
    data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B19301_001E", "B17001_002E", "B17001_002E")
                       , {'for': 'County:*'})
    df= pd.DataFrame(data)
    df = df.rename(columns={"B01003_001E": "Population",
                            "B01002_001E": "Median Age",
                            "B19013_001E": "Household Income",
                            "B19301_001E": "Per Capita Income",
                            "B17001_002E": "Poverty Count",
                            "NAME": "Name", "zip code tabulation area": "Zipcode"})
    df["census_year"]= year
    census_data.append(df)

In [ ]:
#Write Census Data into CSVs in Resources
for df in census_data:
    df.to_csv(f"./Resources/CensusByCounty/CountyCensus{df['census_year'][0]}.csv")

In [ ]:
#Read in csv for school county data
schools = pd.read_csv('./Resources/ZipsCountiesForSchools.csv')

In [ ]:
bottom10 = schools[['County in Census']][0:10].values.tolist()
top10 = schools[['County in Census']][10:20].values.tolist()

In [ ]:
bottom10df = pd.DataFrame()
top10df = pd.DataFrame()

In [ ]:
test = census_data[0]

In [ ]:
#Top10
for df in census_data:
    for i in range(len(top10)):
        top10df = top10df.append(df[df['Name']==top10[i][0]])

In [ ]:
#Bottom10
for df in census_data:
    for i in range(len(bottom10)):
        bottom10df = bottom10df.append(df[df['Name']==bottom10[i][0]])

In [ ]:
top10df_indexed = top10df.set_index(['census_year','Name'])
bottom10df_indexed = bottom10df.set_index(['census_year','Name'])

In [ ]:
#2015 data did not include Washington, DC, so District of Columbia needed to be dropped to properly calcuate the difference.
top10delta_pcincome = top10df_indexed.loc[2015]['Per Capita Income'] - top10df_indexed.loc[2009]['Per Capita Income']
top10delta_pcincome = top10delta_pcincome.drop(labels = "District of Columbia")
bottom10delta_pcincome = bottom10df_indexed.loc[2015]['Per Capita Income'] - bottom10df_indexed.loc[2009]['Per Capita Income']
bottom10delta_pcincome = bottom10delta_pcincome.drop(labels = "District of Columbia")

# top10delta_hhincome = top10df_indexed.loc[2015]['Per Capita Income'] - top10df_indexed.loc[2009]['Per Capita Income']
# top10delta_hhincome = top10delta_hhincome.drop(labels = "District of Columbia")
# bottom10delta_hhincome = bottom10df_indexed.loc[2015]['Per Capita Income'] - bottom10df_indexed.loc[2009]['Per Capita Income']
# bottom10delta_hhincome = bottom10delta_hhincome.drop(labels = "District of Columbia")

# top10delta_pop = top10df_indexed.loc[2015]['Per Capita Income'] - top10df_indexed.loc[2009]['Per Capita Income']
# top10delta_pop = top10delta_pcincome.drop(labels = "District of Columbia")
# bottom10delta_pop = bottom10df_indexed.loc[2015]['Per Capita Income'] - bottom10df_indexed.loc[2009]['Per Capita Income']
# bottom10delta_pop = bottom10delta_pcincome.drop(labels = "District of Columbia")




In [ ]:
data = [top10delta_pcincome, bottom10delta_pcincome]
plt.boxplot(data)
plt.xticks([1, 2], ['Top10', 'Bottom10'])
plt.ylabel('Dollars')
plt.title('Per Capita Income Change from 2009 - 2015')

plt.savefig("./Images/PCIncomeChange")

plt.show()

In [ ]:
for city in top10:
    plt.plot(top10df[top10df['Name']==city[0]]['census_year'],
             top10df[top10df['Name']==city[0]]['Per Capita Income'])


plt.legend(top10,prop={'size': 6})

plt.show()

In [ ]:
for city in bottom10:
    plt.plot(bottom10df[bottom10df['Name']==city[0]]['census_year'],
             bottom10df[bottom10df['Name']==city[0]]['Per Capita Income'])

plt.legend(bottom10,prop={'size': 6})
    
plt.show()